<a href="https://colab.research.google.com/github/varshiarjampudi/FMML__labs__and__Projects/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Yes, averaging validation accuracy across multiple splits can provide more consistent results. This technique is known as:


*K-Fold Cross-Validation*


Benefits:


1. Reduces overfitting: Averages out variability in validation sets.
2. Improves generalization: Evaluates model performance on multiple subsets.
3. Increases reliability: Provides a more robust estimate of model accuracy.


How it works:


1. Divide dataset into k subsets (folds).
2. Train model on k-1 folds, validate on remaining fold.
3. Repeat step 2 for each fold.
4. Calculate average validation accuracy across all folds.


*Advantages:*

1. More accurate estimation of model performance.
2. Reduces impact of lucky or unlucky splits.
3. Helps hyperparameter tuning.


*Common values for k:*

1. k=5 (5-fold cross-validation)
2. k=10 (10-fold cross-validation)


*When to use:*

1. Small datasets: Reduces overfitting.
2. Imbalanced datasets: Helps evaluate model performance.
3. Hyperparameter tuning: Provides robust estimates.


*Code examples:*

Python (scikit-learn):
```
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

avg_accuracy = np.mean(accuracies)
```

R (caret):
```
library(caret)
train_control <- trainControl(method = "cv", number = 5)
model <- train(x, y, method = "rf", trControl = train_control)
```

Would you like more information on cross-validation or hyperparameter tuning?

Yes, averaging validation accuracy across multiple splits (k-fold cross-validation) provides a more accurate estimate of test accuracy.


_Why:_

1. Reduces variability: Averages out fluctuations in validation sets.
2. Better representation: Evaluates model performance on multiple subsets.
3. Less dependent on single split: Reduces impact of lucky or unlucky splits.


_Benefits:_

1. Improved generalization estimate
2. More reliable model comparison
3. Better hyperparameter tuning


_Conditions for accuracy:_

1. Sufficient k: Increase k for smaller datasets.
2. Representative folds: Ensure folds resemble test data.
3. Independent validation: Keep validation sets separate.


_Comparison to test accuracy:_

1. Close approximation: K-fold CV accuracy ≈ test accuracy.
2. Conservative estimate: K-fold CV accuracy ≤ test accuracy.


_When to trust k-fold CV accuracy:_

1. Large k (e.g., k=10).
2. Small standard deviation across folds.
3. Model stability across hyperparameters.


_Additional considerations:_

1. Stratified k-fold: Preserve class balance.
2. Repeated k-fold: Average multiple runs.


To further improve estimates:


1. Use stratified k-fold.
2. Increase k.
3. Employ repeated k-fold.
4. Monitor standard deviation.


Would you like more information on:


1. Stratified k-fold?
2. Repeated k-fold?
3. Hyperparameter tuning?
4. Model evaluation metrics?

The number of iterations (k) in k-fold cross-validation affects the estimate's precision and reliability.


*Effects of increasing k:*

1. *Reduced variability*: More iterations reduce fluctuations in validation scores.
2. *Improved accuracy*: Better estimate of test accuracy.
3. *Increased computational cost*: More iterations require more training and validation.


*Benefits of higher k:*

1. *More reliable estimate*: Averages out lucky/unlucky splits.
2. *Better model comparison*: Compares models more accurately.
3. *Improved hyperparameter tuning*: Optimizes hyperparameters more effectively.


*Diminishing returns:*

1. *Limited improvement*: Beyond k=10, gains diminish.
2. *Computational cost*: Higher k increases training time.


*Optimal k:*

1. *k=5 or k=10*: Common choices for balancing accuracy and computational cost.
2. *Dataset size*: Larger datasets may require smaller k.


*Considerations:*

1. *Computational resources*: Balance accuracy and training time.
2. *Model complexity*: Simpler models may require fewer iterations.
3. *Dataset characteristics*: Imbalanced datasets may benefit from stratified k-fold.


*Visualization:*

| k  | Variability | Accuracy | Computational Cost |
|----|-------------|----------|--------------------|
| 2  | High        | Low      | Low                |
| 5  | Medium      | Medium   | Medium             |
| 10 | Low         | High     | High               |
| 20 | Very Low    | Very High| Very High          |


To determine the optimal k for your specific problem:


1. Monitor validation score variability.
2. Evaluate computational resources.
3. Consider dataset characteristics.


Would you like more information on:


1. Stratified k-fold?
2. Repeated k-fold?
3. Hyperparameter tuning?
4. Model evaluation metrics?

Increasing iterations (k) in k-fold cross-validation can help alleviate issues with small train or validation datasets, but it's not a definitive solution.


Benefits:


1. Reduced variability: More iterations average out fluctuations.
2. Improved estimate: Better representation of model performance.


Limitations:


1. Overfitting: Small datasets may still lead to overfitting.
2. Computational cost: Higher k increases training time.


When dealing with very small datasets:


1. k-fold CV helps, but may not eliminate issues.
2. Consider data augmentation (if applicable).
3. Regularization techniques (e.g., dropout, L1/L2 reg).


Minimum dataset size guidelines:


1. Classification: 5-10 samples per class.
2. Regression: 10-20 samples per feature.


If dataset size is:


1. <5 samples per class: Consider data augmentation or transfer learning.
2. 5-10 samples per class: Use k-fold CV with caution.
3. 10+ samples per class: K-fold CV is more reliable.


To mitigate small dataset issues:


1. Collect more data (if possible).
2. Use transfer learning or pre-trained models.
3. Employ regularization techniques.
4. Consider Bayesian methods or semi-supervised learning.


Keep in mind:


1. K-fold CV is not a substitute for sufficient data.
2. Small datasets may require specialized techniques.


Would you like more information on:


1. Data augmentation?
2. Regularization techniques?
3. Transfer learning?
4. Bayesian methods?
5. Semi-supervised learning?